### Load packages and sample data

In [1]:
import shlex
import re
import pandas as pd
import numpy as np
from typing import List
import utils

In [2]:
fields_to_include = ["OWNER","TITLE", "ABSTRACT", "SERVICETYPE", "SERVICELINK", "KEYWORDS"]
fields_to_output = ["OWNER","TITLE", "SERVICETYPE", "SERVICELINK"]
url_github_repo = "https://github.com/davidoesch/geoservice_harvester_poc/blob/main/data/"
url_github_repo_suffix = "?raw=true"
url_geoservices_CH_csv = "{}geoservices_CH.csv{}".format(url_github_repo,url_github_repo_suffix)
a = 'Strassen auswertung system'

In [3]:
def split_delimiters(word_list_with_delimiters: List[str]) -> List[str]:
    """Take care of left over delimiters, split strings even if in qoutes
        Return a list of words """
    delimiters = [";", ","]

    new_word_list = []

    for word in word_list_with_delimiters:
        if (any(delimiter in word for delimiter in delimiters)):
            splitted_words = re.split(r',|;', word)
            for splitted_word_ in splitted_words:
                new_word_list.append(splitted_word_)
        else:
            new_word_list.append(word)
    return new_word_list

def load_data():
    dataframe = pd.read_csv(url_geoservices_CH_csv, usecols=fields_to_include)
    return dataframe

In [4]:
res = split_delimiters(shlex.split(a))
dataframe_some_cols = load_data()
result = dataframe_some_cols[dataframe_some_cols.apply(lambda dataset: dataset.astype(str).str.contains(res[0], case=False).any(), axis=1)]
result =result.fillna("")

### TF-IDF ranking of the abstract

In [5]:
result['tfidf'] = result.apply(lambda row: utils.ranking_tfidf(row['ABSTRACT']), axis=1)


In [18]:
result.loc[690]['ABSTRACT'], result.loc[690]['tfidf']

('Innerortsstrecken (IO) auf den Kantonsstrassen gemäss Strassengesetzt (SAR 751.100). Die Datenhaltung erfolgt im ESRI Modul "Roads&Highways". Innerortsstrecken (IO) auf den Kantonsstrassen gemäss Strassengesetzt (SAR 751.100). Die Datenhaltung erfolgt im ESRI Modul "Roads&Highways". None',
 ([('none', 1.0),
   ('datenhaltung', 0.82),
   ('erfolgt', 0.82),
   ('esri', 0.82),
   ('highway', 0.82),
   ('modul', 0.82),
   ('road', 0.82),
   ('100', 0.71),
   ('751', 0.71),
   ('gemäss', 0.71),
   ('innerortsstrecken', 0.71),
   ('io', 0.71),
   ('kantonsstrassen', 0.71),
   ('sar', 0.71),
   ('strassengesetzt', 0.71)],
  'german'))

In [26]:
result.loc[20123]['ABSTRACT'], result.loc[20123]['tfidf'], result.loc[20123]['KEYWORDS'], result.loc[20123]['TITLE']

('Das Strassennetz als Teil des Topic Transportation des swissTLMRegio Produktes von swisstopo beschreibt die verschiedenen Elemente im Zusammenhang mit dem Verkehr / Transport. Es gibt Informationen über: Zollämter (CustomsOffice), wichtige Strassenknoten (Interchange), Auffahrtsrampen (Ramp), Strassen und Wege (Road), Autofähren (Ship) und Verkehrsinformationen zu den Strassen (TrafficInfo). Das detaillierte Verkehrsregime (Abbiegeverbote, Einbahnstrassen, usw.) ist nicht enthalten. Öffentliche Daten der Bundes Geodaten-Infrastruktur (BGDI) (Revision: wms-bgdi: 974b958 mapfile_include: 40688b8)',
 ([('bgdi', 0.53),
   ('transport', 0.52),
   ('strassen', 0.46),
   ('abbiegeverbot', 0.41),
   ('detailliert', 0.41),
   ('einbahnstrassen', 0.41),
   ('enthalten', 0.41),
   ('usw', 0.41),
   ('verkehrsregim', 0.41),
   ('40688b8', 0.27),
   ('974b958', 0.27),
   ('bund', 0.27),
   ('daten', 0.27),
   ('geodaten', 0.27),
   ('infrastruktur', 0.27),
   ('mapfile_includ', 0.27),
   ('revis'

### Improvement of the TF-IDF with BM25

In [4]:
# abstract = result.loc[20123]['ABSTRACT']
abstract = "Das Strassennetz als Teil des Topic Transportation des swissTLMRegio Produktes von swisstopo beschreibt die verschiedenen Elemente im Zusammenhang mit dem Verkehr / Transport. Es gibt Informationen über: Zollämter (CustomsOffice), wichtige Strassenknoten (Interchange), Auffahrtsrampen (Ramp), Strassen und Wege (Road), Autofähren (Ship) und Verkehrsinformationen zu den Strassen (TrafficInfo). Öffentliche Daten der Bundes Geodaten-Infrastruktur (BGDI)"
utils.rank_results_lsi(abstract, split_delimiters(shlex.split(a)))

AttributeError: 'list' object has no attribute 'lower'

In [8]:
scores = np.asarray(query.sum(axis=0)).ravel()
scores

array([1., 1., 1.])

In [9]:
scores = np.asarray(doc.sum(axis=0)).ravel()
scores

array([0.20412415, 0.19245009, 0.19245009, 0.20412415, 0.37796447,
       0.37796447, 0.19245009, 0.20412415, 0.37796447, 0.20412415,
       0.19245009, 0.37796447, 0.40824829, 0.20412415, 0.20412415,
       0.19245009, 0.37796447, 0.19245009, 0.20412415, 0.19245009,
       0.37796447, 0.19245009, 0.20412415, 0.20412415, 0.19245009,
       0.19245009, 0.19245009, 0.38490018, 0.19245009, 0.20412415,
       0.20412415, 0.20412415, 0.20412415, 0.20412415, 0.19245009,
       0.20412415, 0.20412415, 0.38490018, 0.20412415, 0.19245009,
       0.20412415, 0.20412415, 0.19245009, 0.19245009, 0.19245009,
       0.19245009, 0.20412415, 0.37796447, 0.19245009])

['road', 'asset', 'management', 'system']